In [19]:
import pickle
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import math
import sys
import csv
import sys, getopt
import json
import os
import numpy as np
import pandas as pd
import seaborn as sns
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random
import pandas as pd
from rake_nltk import Rake
#pip install rake-nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#Configuración de la conexión a Mysql
try:
    cnx = mysql.connector.connect(user='user_taller3', password='taller3.', host='127.0.0.1', database='taller3')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
      print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
      print("Database does not exist")
    else:
      print(err)

cursor = cnx.cursor()
# Print arguments one by one
#print ('First argument:',  str(argv))
file_path = './movies_bagofwords.csv'
df=pd.read_csv(file_path, sep = ',', header=0, names = ["movieId","title","bagofwords"] )
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bagofwords'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim 


array([[1.        , 0.        , 0.02597622, ..., 0.02360961, 0.        ,
        0.02553215],
       [0.        , 1.        , 0.02418254, ..., 0.02197935, 0.02721655,
        0.        ],
       [0.02597622, 0.02418254, 1.        , ..., 0.03189099, 0.01974496,
        0.01724394],
       ...,
       [0.02360961, 0.02197935, 0.03189099, ..., 1.        , 0.01794606,
        0.0156729 ],
       [0.        , 0.02721655, 0.01974496, ..., 0.01794606, 1.        ,
        0.        ],
       [0.02553215, 0.        , 0.01724394, ..., 0.0156729 , 0.        ,
        1.        ]])

In [13]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):

    # initializing the empty list of recommended movies
    recommended_movies = []
    try:
        # gettin the index of the movie that matches the title
        idx = indices[indices == title].index[0]

        # creating a Series with the similarity scores in descending order
        score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

        # getting the indexes of the 10 most similar movies
        top_10_indexes = list(score_series.iloc[1:11].index)

        # populating the list with the titles of the best 10 matching movies
        for i in top_10_indexes:
            recommended_movies.append(list(df.index)[i])
    except:
        i=0
        #print("not recomend")
    return recommended_movies

In [14]:
#user_id= str(argv)
user_id = 162543
sql_artists = "SELECT user_id,movie_id,score as rating from preferences where user_id ="+ user_id
#cursor.execute(sql_artists)
ratings = pd.DataFrame()
ratings = ratings.append(pd.read_sql(sql_artists, cnx))
print(df)

# load the model from disk
import pickle
filename = 'filtrado_colaborativo.sav'
loaded_model = pickle.load(open(filename, 'rb'))

movies= pd.read_csv('ml-latest-small/movies.csv')
unique_movies = movies.movieId.unique().shape[0]
unique_users = 1
matrix_all = np.zeros((unique_users, unique_movies))
movies['movie_id_simple'] = pd.factorize(movies.moviesId)[0]

sql = "INSERT INTO `recomendations_movies` (`user_id`, `movie_id`, `recomendation_score`) VALUES (%s, %s, %s)"

for i in range(0,unique_users):
    #print(user_id)
    for k in ratings.index:
        business_id = current_reviews['movieId'][k]
        #print(business_id)
        list_rec = recommendations(business_id)
        if len(list_rec) >0:
            for k in list_rec:
                #cursor.execute(sql, (str(user_id),str(k),5))
        cnx.commit()



IndentationError: expected an indented block (<ipython-input-14-2d31fad44287>, line 31)

In [20]:
file_path = './movies_bagofwords.csv'
df=pd.read_csv(file_path, sep = ',', header=0, names = ["movieId","title","bagofwords"] )
movies=movies.merge(df,on = 'movieId',how = 'inner')
movies = movies [['movieId','title','genres']]
movies

,movieId,title,genres
0,42,Dead Presidents (1995),Action|Crime|Drama
1,44,Mortal Kombat (1995),Action|Adventure|Fantasy
2,63,Don't Be a Menace to South Central While Drink...,Comedy|Crime
3,68,French Twist (Gazon maudit) (1995),Comedy|Romance
4,79,"Juror, The (1996)",Drama|Thriller
...,...,...,...
482,172825,Adventures of Mowgli: The Kidnapping (1968),Adventure|Animation|Children
483,174727,Good Time (2017),Crime|Drama
484,175569,Wind River (2017),Action|Crime|Mystery|Thriller
485,182293,Hare-um Scare-um (1939),Animation|Children|Comedy


## insert only movies that have bagofwords

In [24]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import math
import sys
import csv
import os
import pandas as pd

#Configuración de la conexión a Mysql
try:
  cnx = mysql.connector.connect(user='user_taller3', password='taller3.', host='127.0.0.1', database='taller3')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

cursor = cnx.cursor()
df_b=movies

#df_explode = df_b.assign(categories = df_b.categories
#                         .str.split(', ')).explode('categories')
#df_explode=df_explode.reset_index()
#df_explode

print("Finish reading file and dtaframes")

# Create a new record
sql = "INSERT INTO `movies` (`movies_id`, `title`, `genres`) VALUES (%s, %s, %s)"


def isNaN(string):
    return string != string
print(df_b.index)
for i in df_b.index: 
  # Execute the query
  var1= df_b['movieId'][i]
  var2= df_b['title'][i]
  var3= df_b['genres'][i]


  try:
    #print(var1,var2,var3)
    cursor.execute(sql, (int(var1),var2,var3))
  except mysql.connector.errors.DataError as err:
    print("Track var 1: "+ var1+ " ")
    sys.exit(1)
  
  # the connection is not autocommited by default. So we must commit to save our changes.
  cnx.commit()


#print(df_artist)
cursor.close()
cnx.close()

Finish reading file and dtaframes
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            477, 478, 479, 480, 481, 482, 483, 484, 485, 486],
           dtype='int64', length=487)


In [ ]:
  send_message_back = {
    'arguments': argv,
    'message': """Hello,
  This is my message.
  To the world"""
  }
  print(json.dumps(send_message_back))
  
if __name__ == "__main__":
    main(sys.argv[1])